# Delta Robot Kinematics, Configuration Space, Workspace and Singularity Analysis

## Degrees of Freedom

The delta robot has 3 degrees of freedom. The 3 degrees of freedom are the 3 angles of the 3 arms. The 3 angles are the angles between the arms and the base. The 3 angles are denoted by $\theta_1$, $\theta_2$, and $\theta_3$. The 3 angles are the joint angles of the delta robot. The 3 angles are the configuration space of the delta robot. The configuration space of the delta robot is the space of all possible joint angles of the delta robot. The configuration space of the delta robot is a 3-dimensional space. The configuration space of the delta robot is a 3-dimensional space because the delta robot has 3 degrees of freedom.

<div style="text-align: center;">
  <img src="images/delta_robot_modern_robotics.png" alt="Delta Robot">
</div>

In [113]:
# Import block
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration Space

The delta robot's configuration space is bound by the 3 motor angles $\theta_1$, $\theta_2$, and $\theta_3$. The configuration space is a 3D space, where each point in the space corresponds to a unique position of the robot's end effector.

In [114]:
# The C-Space for the delta robot is 3 dimensional for the 3 motors

# Motor ranges in Dynamixel steps
theta1_range = [2100, 2850]
theta2_range = [2100, 2850]
theta3_range = [2100, 2850]

# Use a smaller number of points for a scatter plot to keep rendering fast
num_points = 20
theta1 = np.linspace(theta1_range[0], theta1_range[1], num_points)
theta2 = np.linspace(theta2_range[0], theta2_range[1], num_points)
theta3 = np.linspace(theta3_range[0], theta3_range[1], num_points)
Theta1, Theta2, Theta3 = np.meshgrid(theta1, theta2, theta3)

# Create a 3D scatter plot of the configuration space points using Plotly
fig = go.Figure(data=[go.Scatter3d(
  x=Theta1.flatten(),
  y=Theta2.flatten(),
  z=Theta3.flatten(),
  mode='markers',
  marker=dict(size=3, opacity=0.5)
)])

# Set the labels
fig.update_layout(scene=dict(
  xaxis_title='Theta1',
  yaxis_title='Theta2',
  zaxis_title='Theta3'
))

# Show the plot
fig.show()

## Robot Constants

In [115]:
# Robot Constants
SB = 261.306  # Base Equilateral Triangle Side Length [mm]
SP = 73.481  # Platform Equilateral Triangle Side Length [mm]
L = 108  # Active Link Length [mm]
ELL = 160  # Passive Link Length [mm]
H = 32  # Passive Link Width [mm]
WB = (np.sqrt(3) / 6) * SB
UB = (np.sqrt(3) / 3) * SB
WP = (np.sqrt(3) / 6) * SP
UP = (np.sqrt(3) / 3) * SP

# Inverse Kinematics
https://people.ohio.edu/williams/html/PDF/DeltaKin.pdf

In [116]:
def IK(x,y,z):
  # The analytical solution for IK uses the constraint equations found from the vector-loop closure equations
  # E_i * cos(theta_i) + F_i *  sin(theta_i) + G_i = 0
  # Variables obtained from the Vector-Loop Closure Equations
  a = WB - UP
  b = (SP / 2) - ((np.sqrt(3) / 2) * WP)
  c = WP - (WB / 2)
  E = (
    2*L*(y+a), # E1
    -L*(np.sqrt(3)*(x+b)+y+c),  # E2
    L*(np.sqrt(3)*(x-b)-y-c)  # E3
    )
  F = (
    2*z*L, # F1
    2*z*L,  # F2
    2*z*L,  # F3
  )
  G = (
      x**2 + y**2 + z**2 + a**2 + L**2 + 2*y*a - ELL**2,  # G1
      x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(x*b+y*c) - ELL**2,  # G2
      x**2 + y**2 + z**2 + b**2 + c**2 + L**2 + 2*(-x*b+y*c) - ELL**2,  # G3
  )
  thetas = []
  # Tangent Half-Angle Substitution
  for i in range(3):
    t_plus = (-F[i] + np.sqrt(E[i]**2 + F[i]**2 - G[i]**2)) / (G[i] - E[i])
    t_minus = (-F[i] - np.sqrt(E[i]**2 + F[i]**2 - G[i]**2)) / (G[i] - E[i])
    theta_plus = 2 * np.arctan(t_plus)
    theta_minus = 2 * np.arctan(t_minus)
    # Pick the solution with the knees "kinked" out, the angle should be closer to 0
    thetas.append(theta_plus if abs(theta_plus) < abs(theta_minus) else theta_minus)
  # return a tuple of the 3 joint angles
  return (thetas[0], thetas[1], thetas[2])


print(IK(0, 0, -200))

(0.8325196632423271, 0.6266561718100904, 0.6266561718100904)


In [117]:
# Plot a simple up and down trajectory
num_samples = 100
z_traj = np.linspace(-180, -280, num_samples)
x_traj = np.zeros(num_samples)
y_traj = np.zeros(num_samples)
# For each (x,y,z) point, calculate the IK solution.
thetas = np.array([IK(x, y, z) for x, y, z in zip(x_traj, y_traj, z_traj)])

# Trajectory: An XY Circle with a center of (0, 0, -200) mm and a radius of 50 mm
# At the same time, the Z displacement goes through 2 cycles of a sine wave
# centered on Z = -200 mm with an amplitude of 25 mm
t = np.linspace(0, 2 * np.pi, num_samples)
x_circle = 50 * np.cos(t)
y_circle = 50 * np.sin(t)
# Two sine cycles in z with amplitude 25 mm
z_circle = -200 + 25 * np.sin(2 * t)
thetas_circle = np.array([IK(x, y, z) for x, y, z in zip(x_circle, y_circle, z_circle)])
# Plot the trajectories
fig = make_subplots(rows=2, cols=1, subplot_titles=('Up and Down Trajectory', 'XY Circle Trajectory'))

# Plot the up and down trajectory
fig.add_trace(go.Scatter(y=thetas[:, 0], mode='lines', name='Theta1'), row=1, col=1)
fig.add_trace(go.Scatter(y=thetas[:, 1], mode='lines', name='Theta2'), row=1, col=1)
fig.add_trace(go.Scatter(y=thetas[:, 2], mode='lines', name='Theta3'), row=1, col=1)

# Plot the XY circle trajectory
fig.add_trace(go.Scatter(y=thetas_circle[:, 0], mode='lines', name='Theta1'), row=2, col=1)
fig.add_trace(go.Scatter(y=thetas_circle[:, 1], mode='lines', name='Theta2'), row=2, col=1)
fig.add_trace(go.Scatter(y=thetas_circle[:, 2], mode='lines', name='Theta3'), row=2, col=1)

fig.update_layout(height=800, width=800, title_text="Trajectories")
fig.show()

/tmp/ipykernel_291248/42305886.py:26: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_291248/42305886.py:27: RuntimeWarning:

invalid value encountered in sqrt



# Forward Kinematics


In [118]:
def virtual_sphere_centers(th1, th2, th3):
  # thetas are the joint angles [rad]
  return (
    (0, -WB-L*np.cos(th1)+UP, -L*np.sin(th1)), # Sphere 1
    ((np.sqrt(3)/2)*(WB+L*np.cos(th2))-(SP/2), (1/2) * (WB+L*np.cos(th2))-WP, -L*np.sin(th2)),  # Sphere 2
    (-(np.sqrt(3)/2)*(WB+L*np.cos(th3))+(SP/2), (1/2) * (WB+L*np.cos(th3))-WP, -L*np.sin(th3)),  # Sphere 3
  )

def FK(th1,th2,th3):
  # First determine the centers of the 3 virtual spheres
  sphere_centers = virtual_sphere_centers(th1, th2, th3)
  x1, y1, z1 = sphere_centers[0]
  x2, y2, z2 = sphere_centers[1]
  x3, y3, z3 = sphere_centers[2]
  
  # Determine if the all Z heights of the 3 spheres are the same to determine which algorithm to use
  same_z_height = all([center[2] == sphere_centers[0][2] for center in sphere_centers])
  
  if same_z_height:
    assert z1 == z2 == z3
    z_n = z1 # All Z heights are the same
    a = 2*(x3 - x1)
    b = 2*(y3 - y1)
    c = ELL**2 - ELL**2 - x1**2 - y1**2 + x3**2 + y3**2
    d = 2*(x3 - x2)
    e = 2*(y3 - y2)
    f = ELL**2 - ELL**2 - x2**2 - y2**2 + x3**2 + y3**2
    denominator = (a*e - b*d)
    x = (c*e - b*f) / denominator
    y = (a*f - c*d) / denominator
    B = -2 * z_n
    C = z_n**2 - ELL**2 + (x - x1)**2 + (y - y1)**2
    discriminant = B**2 - 4*C
    z_plus = (-B + np.sqrt(discriminant)) / 2
    z_minus = (-B - np.sqrt(discriminant)) / 2
    # Choose the Z value that is in the correct workspace (-Z)
    z = z_plus if z_plus < 0 else z_minus
    return x,y,z
  else:
    # 3 Sphere Intersection Algorithm
    pass


# Test FK with equal joint angles which should result in equal Z heights
print(FK(-0.5, -0.5, -0.5))

(1.027288075989331e-15, 4.1517353304541306e-15, -44.50514016116509)


# Plotting the Robot's Configuration

In order to better visualize the robot's configuration, we can plot the robot's links and joints in 3D space.

In [120]:
def motor_positions():
  return (
    (0,-WB,0), # B1
    ((np.sqrt(3)/2)*WB,WB/2,0), # B2
    (-(np.sqrt(3)/2)*WB, WB/2, 0)  # B3
  )
  
def base_vertices():
  return (
    (SB/2, -WB, 0), # b1
    (0, UB, 0), # b2
    (-SB/2, -WB, 0) # b3
  )

def platform_vertices(platform_position):
  # platform_position is a tuple (x,y,z)
  x, y, z = platform_position
  # Platform vertices with respect to the given platform position
  return (
    (x, y - UP, z), # P1
    (x + SP/2, y + WP, z), # P2
    (x - SP/2, y + WP, z) # P3
  )

def knee_joints(theta1,theta2,theta3):
  knee_1 = (0, -WB-L*np.cos(theta1), -L*np.sin(theta1))
  knee_2 = ((np.sqrt(3)/2)*(WB+L*np.cos(theta2)), (1/2)*(WB+L*np.cos(theta2)), -L*np.sin(theta2))
  knee_3 = ((-np.sqrt(3)/2)*(WB+L*np.cos(theta3)), (1/2)
            * (WB+L*np.cos(theta3)), -L*np.sin(theta3))
  return (knee_1, knee_2, knee_3)


def get_motor_rectangle_corners(motor_position, angle, motor_width=20, motor_length=40):
    # Local coordinates (centered at 0,0)
    dx = motor_length / 2
    dy = motor_width / 2
    local_corners = np.array([
        [-dx, -dy],
        [dx, -dy],
        [dx,  dy],
        [-dx,  dy]
    ])
    # Create rotation matrix for the given angle
    R = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])
    rotated = (local_corners @ R.T) + np.array(motor_position[:2])
    # Append the constant z coordinate from motor_position
    corners = [(pt[0], pt[1], motor_position[2]) for pt in rotated]
    return corners

def plot_robot(theta1, theta2, theta3):
  # Calculate the knee joint positions
  knee_1, knee_2, knee_3 = knee_joints(theta1, theta2, theta3)
  # Calculate the platform position
  platform_position = FK(theta1, theta2, theta3)
  # Get the base vertices
  bv1, bv2, bv3 = base_vertices()
  # Get the motor positions
  m1, m2, m3 = motor_positions()
  # Get the platform vertices
  pv1, pv2, pv3 = platform_vertices(platform_position)
  
  # Create a 3D plot
  fig = go.Figure()
  
  # --- Base Triangle (Mesh3d) ---
  base_x = [bv1[0], bv2[0], bv3[0]]
  base_y = [bv1[1], bv2[1], bv3[1]]
  base_z = [bv1[2], bv2[2], bv3[2]]
  fig.add_trace(go.Mesh3d(
      x=base_x, y=base_y, z=base_z,
      i=[0], j=[1], k=[2],
      color='lightblue', opacity=0.5,
      name='Base'
  ))
  
  # --- Platform Triangle (Mesh3d) ---
  plat_x = [pv1[0], pv2[0], pv3[0]]
  plat_y = [pv1[1], pv2[1], pv3[1]]
  plat_z = [pv1[2], pv2[2], pv3[2]]
  fig.add_trace(go.Mesh3d(
      x=plat_x, y=plat_y, z=plat_z,
      i=[0], j=[1], k=[2],
      color='lightgreen', opacity=0.5,
      name='Platform'
  ))

  # Add platform vertices as markers with labels
  platform_vertices_list = [pv1, pv2, pv3]
  for idx, pv in enumerate(platform_vertices_list, start=1):
      fig.add_trace(go.Scatter3d(
          x=[pv[0]], y=[pv[1]], z=[pv[2]],
          mode='markers+text',
          marker=dict(color='green', size=H/2),
          text=f'P{idx}',
          textposition='top center',
          name=f'PV {idx}',
          showlegend=False
      ))
  
  # --- Motor Rectangles ---
  # Determine motor orientations based on the base triangle sides:
  motor_angle1 = np.arctan2(bv1[1] - bv3[1], bv1[0] - bv3[0])
  motor_angle2 = np.arctan2(bv2[1] - bv1[1], bv2[0] - bv1[0])
  motor_angle3 = np.arctan2(bv3[1] - bv2[1], bv3[0] - bv2[0])
  motor_angles = [motor_angle1, motor_angle2, motor_angle3]
  motor_positions_list = [m1, m2, m3]
  motor_colors = ['red', 'green', 'blue']
  motor_ids = ['1', '2', '3']
  
# For each motor, compute its rectangle and add it as two triangles.
  for m, ang, color, mid in zip(motor_positions_list, motor_angles, motor_colors, motor_ids):
    corners = get_motor_rectangle_corners(m, ang)
    # Split quadrilateral into two triangles:
    # Triangle 1: corners[0], corners[1], corners[2]
    tri1_x = [corners[0][0], corners[1][0], corners[2][0]]
    tri1_y = [corners[0][1], corners[1][1], corners[2][1]]
    tri1_z = [corners[0][2], corners[1][2], corners[2][2]]
    
    # Triangle 2: corners[0], corners[2], corners[3]
    tri2_x = [corners[0][0], corners[2][0], corners[3][0]]
    tri2_y = [corners[0][1], corners[2][1], corners[3][1]]
    tri2_z = [corners[0][2], corners[2][2], corners[3][2]]
    
    # Add first triangle
    fig.add_trace(go.Mesh3d(
        x=tri1_x, y=tri1_y, z=tri1_z,
        color=color, opacity=0.7,
        name=f'Motor {mid}',
        showscale=False
    ))
    # Add second triangle
    fig.add_trace(go.Mesh3d(
        x=tri2_x, y=tri2_y, z=tri2_z,
        color=color, opacity=0.7,
        name=f'Motor {mid}',
        showscale=False
    ))
    # Add motor label as text at the motor position
    fig.add_trace(go.Scatter3d(
        x=[m[0]], y=[m[1]], z=[m[2]],
        mode='text',
        text=[mid],
        textposition='middle center',
        textfont=dict(color='black', size=18),
        showlegend=False
    ))
  # --- End of Motor Rectangles ---
    
  # --- Active Links (lines from motors to knee joints) ---
  knees = [knee_1, knee_2, knee_3]
  for m, k in zip(motor_positions_list, knees):
      fig.add_trace(go.Scatter3d(
          x=[m[0], k[0]],
          y=[m[1], k[1]],
          z=[m[2], k[2]],
          mode='lines',
          line=dict(color='purple', width=H/4),
          name='Active Link' + str(motor_positions_list.index(m) + 1),
          showlegend=False
      ))

      # --- Knee Joints as markers ---
      fig.add_trace(go.Scatter3d(
          x=[k[0]], y=[k[1]], z=[k[2]],
          mode='markers',
          marker=dict(color='black', size=10),
          name='Knee Joint' + str(knees.index(k) + 1),
          showlegend=False
      ))

  # --- Passive Links (rectangles from knee joints to platform vertices) ---
  for k, p in zip(knees, [pv1, pv2, pv3]):
      fig.add_trace(go.Scatter3d(
          x=[k[0], p[0]],
          y=[k[1], p[1]],
          z=[k[2], p[2]],
          mode='lines',
          line=dict(color='orange', width=H),
          name='Passive Link' + str(knees.index(k) + 1),
          showlegend=False
      ))
  
  
  # --- Update Layout and Style Plot ---
  fig.update_layout(
      scene=dict(
          xaxis_title='X [mm]',
          yaxis_title='Y [mm]',
          zaxis_title='Z [mm]',
          xaxis=dict(range=[-200, 200]),
          yaxis=dict(range=[-200, 200]),
          zaxis=dict(range=[-400, 0])
      ),
      title='Delta Robot Configuration',
      width=1400,
      height=800
  )
  
  fig.show()

plot_robot(
  theta1=np.pi/4,
  theta2=np.pi/4,
  theta3=np.pi/4
)